In [14]:
# Import all the necessary packages

import scipy as sc
import numpy as np

In [40]:
# Define all the functions needed

def systemofeq(par,deltas):
    # Summary: this function is needed to compute the solution of the baseline representative agent model
    # Input: deltas; a guess for the solution of the model
    #        par: a dictionary containing the parameters of the baseline model
    # Output: eqs; a 4x1 vector that measures the distance to the solution (the solution needs to satisfy four equations)
    delta_k = deltas[0];
    delta_kz = deltas[1];
    delta_ck = deltas[2];
    delta_cz = deltas[3];
    barr = (1-par.get("beta"))/par.get("beta");
    bark = ((1-par.get("beta")+par.get("delta")*par.get("beta"))/(par.get("beta")*par.get("alpha")))**(1/(par.get("alpha")-1));
    barw = (1-par.get("alpha"))*(bark**par.get("alpha"));
    ybar = bark**par.get("alpha");

    eqs = np.zeros(4);
    eqs[0]=bark*delta_k - bark/par.get("beta") + barw*delta_ck + bark*delta_ck/par.get("beta");
    eqs[1]=bark**par.get("alpha") - barw*delta_cz - bark*delta_cz/par.get("beta") - bark*delta_kz;
    eqs[2]=par.get("beta")*(1-par.get("alpha"))*(barr+par.get("delta"))*delta_k + par.get("nu")*delta_ck*delta_k-par.get("nu")*delta_ck;
    eqs[3]=par.get("nu")*delta_ck*delta_kz + par.get("nu")*delta_cz*par.get("rho") + par.get("nu")*delta_cz + par.get("beta")*(barr+par.get("delta"))*par.get("rho") - par.get("beta")*(1-par.get("alpha"))*(barr+par.get("delta"))*delta_kz;
    return eqs


def SimulateData(par,T):
    # Summary: this function simulates data from the baseline representative agent mdodel
    # Input: par; a dictionary containing the parameters of the baseline model
    #        T; the number of datapoints you want to simulate
    # Output: logk, logr, logc, logy:   Tx1 vectors with data simulated from the model
    #         log k is the natural logarithm of capital, log r of interest rates, log c of consumption, log y of output
    
    #First: solve the model
    deltainit = 0.01*np.ones(4);
    fun = lambda x: systemofeq(par,x)
    deltasol = sc.optimize.root(fun,deltainit); 
    deltassol=deltasol.x;
    delta_k = deltassol[0];
    delta_kz = deltassol[1];
    delta_ck = deltassol[2];
    delta_cz = deltassol[3];
    # for the model solution, simulate data
    rbar = (1-par.get("beta"))/par.get("beta");
    kbar = ((1-par.get("beta")+par.get("delta")*par.get("beta"))/(par.get("beta")*par.get("alpha")))**(1/(par.get("alpha")-1));
    wbar = (1-par.get("alpha"))*kbar**par.get("alpha");
    ybar = kbar**par.get("alpha");
    cbar = wbar + kbar/par.get("beta");
    
    eps  = np.sqrt(par.get("s2"))*np.random.randn(T,1);
    
    z = np.zeros(T);
    k = np.zeros(T);
    c = np.zeros(T);
    y = np.zeros(T);
    r = np.zeros(T);
    for t in range(T):
        if t==0:
            z[t] = eps[t];
            k[t] = 0;
        else:
            z[t] = z[t-1]*par.get("rho") + eps[t];
            k[t] = delta_k*k[t-1]+delta_kz*z[t-1];
        
        c[t] = delta_ck*k[t]+delta_cz*z[t];
        y[t] = z[t] + par.get("alpha")*k[t];
        r[t] = ((rbar+par.get("delta"))/rbar)*z[t] + (par.get("alpha")-1)*((rbar+par.get("delta"))/rbar)*k[t];
    
    logk = np.log(kbar) + k ;
    logc = np.log(cbar) + c ;
    logy = np.log(ybar) + y ;
    logr = np.log(rbar) + r ;
    return logk, logr, logc, logy

In [41]:
# Choose a parameter vector you want to simulate data for
par = {"rho":0.7, "s2":0.011, "alpha":0.3, "beta":0.97, "delta":0.1, "nu":3}
# Choose number of data points you want to simulate
T = 100
# Call function for simulation
[logk, logr, logc, logy] = SimulateData(par,T)